In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import json
with open("../data/VLSP 2025 - MLQA-TSR Data Release/law_db/vlsp2025_law.json", "r", encoding="utf-8") as f:
    law_db = json.load(f)

print(law_db[0]['articles'][11])

{'id': '12', 'text': '12.1. Quy chuẩn này quy định các thông số về kích thước biển, hình vẽ trong biển và chữ viết tương ứng với đường đô thị có hệ số là 1 (xem Hình 1 và Bảng 1). Trường hợp đặc biệt, kích thước biển có thể được điều chỉnh để đảm bảo cân đối, đáp ứng tính thẩm mỹ và rõ ràng thông tin.\n12.2. Đối với các đường khác, kích thước biển, hình vẽ trong biển và chữ viết phải nhân với hệ số tương ứng trong Bảng 2, kích thước biển được làm tròn theo nguyên tắc:\na) Số hàng đơn vị ≤ 5 thì lấy bằng 5;\nb) Số hàng đơn vị > 5 thì lấy bằng 0 và tăng số hàng chục lên 1 đơn vị.\n\n<<IMAGE: image002.jpg /IMAGE>>\nHình 1 - Kích thước các loại biển báo\nBảng 1 - Kích thước cơ bản của biển báo hệ số 1\nĐơn vị tính: mm\n<<TABLE: <table border="0" cellpadding="0" cellspacing="0" class="MsoNormalTable" style="width:100.0%;border-collapse:collapse;mso-padding-alt:0cm 0cm 0cm 0cm" width="100%">\n<tr style="mso-yfti-irow:0;mso-yfti-firstrow:yes">\n<td style="width:18.16%;border-top:solid black 1

In [11]:
import re
import pandas as pd
from pandas import DataFrame as df
articles = []
table_pattern = r"<<TABLE:\s*(.*?)\s*/TABLE>>"
image_pattern = r"<<IMAGE:\s*(.*?)\s*/IMAGE>>"
for law in law_db:
    law_id = law['id']
    law_title = law['title']
    for article in law['articles']:
        article_id = article['id']
        article_index = f"{law_id}#{article_id}"
        text = article['text']
        tables_html = re.findall(table_pattern, text, flags=re.S)
        text = re.sub(table_pattern, "", text, flags=re.S).strip()
        images = re.findall(image_pattern, text, flags=re.S)
        text = re.sub(image_pattern, "", text, flags=re.S).strip()
        articles.append({
            "law_id": law_id,
            "article_id": article_index,
            "text": text,
            "tables_html": tables_html,
            "images": images
        })


In [12]:
print(articles[12]["images"])

['image004.jpg']


In [13]:
print(articles[12]["text"])

13.1. Chữ viết trên biển phù hợp với quy định về kiểu chữ nêu tại Phụ lục K của Quy chuẩn này, trường hợp đặc biệt có thể điều chỉnh cho cân đối, phù hợp và đáp ứng thông tin rõ ràng, trong đó:
13.1.1. Sử dụng kiểu chữ tiêu chuẩn “gt1 - Kiểu chữ nén” và “gt2 - Kiểu chữ thường” để ghi thông tin bằng chữ trên biển.
13.1.2. Chữ viết hoa kiểu chữ thường hoặc kiểu chữ nén sử dụng để viết các thông tin chỉ dẫn về hướng đi, các danh từ riêng hoặc các thông tin có tính chất nhấn mạnh, gây chú ý cho người tham gia giao thông. Chỉ nên sử dụng kiểu chữ nén trong trường hợp phải hạn chế kích thước của biển.
13.1.3. Chữ viết thường thường được sử dụng để viết tên địa danh bằng tiếng Anh, các thông tin dịch vụ và trên các biển phụ.
13.1.4. Trên cùng một hàng chữ chỉ sử dụng một loại nét chữ.
13.1.5. Chữ viết trên biển phải là tiếng Việt đủ dấu. Khoảng cách giữa các chữ cái từ 25% - 40% chiều cao chữ, khoảng cách giữa các chữ bất kỳ trên cùng một hàng chữ từ 75% - 100% chiều cao chữ. Khoảng cách theo

In [16]:
print(articles[12]["article_id"])

QCVN 41:2024/BGTVT#13


In [21]:
import re
import pandas as pd
from io import StringIO

articles = []

table_pattern = r"<<TABLE:\s*(.*?)\s*/TABLE>>"
image_pattern = r"<<IMAGE:\s*(.*?)\s*/IMAGE>>"

for law in law_db:
    law_id = law['id']
    law_title = law.get('title', '')  # fallback if title not present

    for article in law['articles']:
        article_id = article['id']
        article_index = f"{law_id}#{article_id}"
        raw_text = article['text']

        # Extract all table HTML blocks
        table_htmls = re.findall(table_pattern, raw_text, flags=re.S)
        images = re.findall(image_pattern, raw_text, flags=re.S)

        # Replace TABLE blocks with markdown
        def table_replacer(match):
            html = match.group(1)
            try:
                dfs = pd.read_html(StringIO(html), header=0 )
                markdown = dfs[0].to_markdown(index=False)
                return "\n\n" + markdown + "\n\n"
            except Exception as e:
                print(f"[ERROR] Failed to parse table in article {article_index}: {e}")
                return "[[TABLE PARSE ERROR]]"

        # Replace all tables
        text_with_markdown_tables = re.sub(table_pattern, table_replacer, raw_text, flags=re.S)

        # Remove image tags from text, but keep their names separately
        text_cleaned = re.sub(image_pattern, "", text_with_markdown_tables, flags=re.S).strip()

        articles.append({
            "law_title": law_title,
            "id": article_index,
            "article_title": article['title'],
            "text": text_cleaned,
            "images": images,
        })


In [22]:
print(articles[11]["images"])

['image002.jpg']


In [23]:
print(articles[11]["text"])

12.1. Quy chuẩn này quy định các thông số về kích thước biển, hình vẽ trong biển và chữ viết tương ứng với đường đô thị có hệ số là 1 (xem Hình 1 và Bảng 1). Trường hợp đặc biệt, kích thước biển có thể được điều chỉnh để đảm bảo cân đối, đáp ứng tính thẩm mỹ và rõ ràng thông tin.
12.2. Đối với các đường khác, kích thước biển, hình vẽ trong biển và chữ viết phải nhân với hệ số tương ứng trong Bảng 2, kích thước biển được làm tròn theo nguyên tắc:
a) Số hàng đơn vị ≤ 5 thì lấy bằng 5;
b) Số hàng đơn vị > 5 thì lấy bằng 0 và tăng số hàng chục lên 1 đơn vị.


Hình 1 - Kích thước các loại biển báo
Bảng 1 - Kích thước cơ bản của biển báo hệ số 1
Đơn vị tính: mm


| Loại  biển             | Kích thước                                               | Độ  lớn   |
|:-----------------------|:---------------------------------------------------------|:----------|
| Biển  tròn             | Đường kính ngoài của biển báo, D                         | 700       |
| Biển  tròn             | Chiều rộng củ

In [24]:
print(articles[11]["law_title"])

QUY CHUẨN KỸ THUẬT QUỐC GIA VỀ BÁO HIỆU ĐƯỜNG BỘ


In [25]:
print(articles[11]["article_title"])

Kích thước của biển báo
